<a href="https://colab.research.google.com/github/minakhan01/PAL_RL/blob/main/Transformers_Computer_Usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Drive

In [ ]:
!pip install -U -q PyDrive
!pip install httplib2==0.15.0
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from pydrive.files import GoogleDriveFileList
from google.colab import auth
from oauth2client.client import GoogleCredentials

from getpass import getpass
import urllib

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

from google.colab import drive as gdrive 
gdrive.mount('/content/gdrive')

DEBUG - Making request: POST https://oauth2.googleapis.com/token
DEBUG - Starting new HTTPS connection (1): oauth2.googleapis.com:443
DEBUG - https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Load data from Drive

In [ ]:
def get_folder_id(drive, parent_folder_id, folder_name):
  """ 
	Check if destination folder exists and return it's ID
	"""
  folder_exists = False

  # Auto-iterate through all files in the parent folder.
  file_list = GoogleDriveFileList()
  try:
    file_list = drive.ListFile(
        {'q': "'{0}' in parents and trashed=false".format(parent_folder_id)}
        ).GetList()
	# Exit if the parent folder doesn't exist
  except googleapiclient.errors.HttpError as err:
  # Parse error message
    message = ast.literal_eval(err.content)['error']['message']
    if message == 'File not found: ':
      exit(1)
  # Exit with stacktrace in case of other error
    else:
      raise

	# Find the the destination folder in the parent folder's files
  for file1 in file_list:
    if file1['title'] == folder_name:
      folder_exists = True
      return file1['id'], folder_exists

  return None, False

In [ ]:
# encoder_model_name = "ViT-B/32"
experiment_id = "MinaData"

folder_name = experiment_id

# Change parentid to match that of experiments root folder in gdrive
parentid = '1fakqJ81EcE65gUtVfFWBagQfGEI9IC1j'

# Initialize sepcific experiment folder in drive
folderid, folder_exists = get_folder_id(drive, parentid, folder_name)
# print(folderid)

WARNING - file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/googleapiclient/discovery_cache/__init__.py", line 44, in autodetect
    from . import file_cache
  File "/usr/local/lib/python3.7/dist-packages

## Read file and redact

In [ ]:
!git clone https://github.com/minakhan01/ActivityWatchRedact

Cloning into 'ActivityWatchRedact'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 71 (delta 24), reused 64 (delta 17), pack-reused 0
Unpacking objects: 100% (71/71), done.


In [ ]:
%cd ActivityWatchRedact
!pwd
!pip install -r requirements.txt

/content/ActivityWatchRedact/ActivityWatchRedact
/content/ActivityWatchRedact/ActivityWatchRedact
Processing /root/.cache/pip/wheels/2b/67/89/414471314a2d15de625d184d8be6d38a03ae1e983dbda91e84/tabulate-0.8.3-cp37-none-any.whl
ERROR: dm-haiku 0.0.4.dev0 has requirement tabulate>=0.8.9, but you'll have tabulate 0.8.3 which is incompatible.
  Found existing installation: tabulate 0.8.9
    Uninstalling tabulate-0.8.9:
      Successfully uninstalled tabulate-0.8.9


In [ ]:
import json
import io
from redact_data import *
from redact_data import _write_file, _load_data, redact_data_basic, redact_data_custom, set_timezone

def get_files_dataset(drive, folderid):
  file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folderid)}).GetList()
  for i, file1 in enumerate(sorted(file_list, key = lambda x: x['title']), start=1):
    print(file1['title'])
    file1.GetContentFile(file1['title'])
    # yield file1['title']

def redact_file(input_file, output_file):

    data, bucket_key_dict = _load_data(input_file)
    redacted_data, unique_urls, unique_app_names = redact_data_basic(data, bucket_key_dict)

    url_blacklist = set(["docs.google.com", "www.deeplearningbook.org"])
    app_blacklist = set(["Finder"])

    user_selected_timezone = 'America/New_York'

    # Remove apps and urls from user provided black list
    redacted_data = redact_data_custom(redacted_data, url_blacklist, app_blacklist, 3)
    
    # set user selected timezone
    redacted_data = set_timezone(redacted_data, user_selected_timezone)

    _write_file(output_file, redacted_data)

get_files_dataset(drive, folderid)
filepath = "aw-buckets-export.json"
output_file = "redacted-data.json"
redact_file(filepath, output_file)
redacted_data, bucket_key_dict = _load_data(output_file)

DEBUG - URL being requested: GET https://www.googleapis.com/drive/v2/files?q=%271Lt5bexsHSnEM7UrC9S8ZT8H9r07IBJnd%27+in+parents+and+trashed%3Dfalse&maxResults=1000&alt=json


aw-buckets-export.json


INFO - AFK data successfully redacted (basic)...
INFO - Web data successfully redacted (basic)...
INFO - Window data successfully redacted (basic)...
INFO - Stopwatch data successfully redacted (basic)...
INFO - Web data successfully redacted (user selected)...
INFO - Window data successfully redacted (user selected)...
INFO - Redacted data written to redacted-data.json


## Process dataset

In [ ]:
import numpy as np
from datetime import datetime

print("bucket_key_dict", bucket_key_dict)
print(type(redacted_data))
# for key in redacted_data["aw-watcher-afk"].keys():
afk = redacted_data["aw-watcher-afk"]
window = redacted_data["aw-watcher-window"]
browser = redacted_data["aw-watcher-web"]
stopwatch = redacted_data["aw-watcher-stopwatch"]
print(redacted_data.keys())
# print(key)
print("window", window["events"][0])
print("afk", afk["events"][0])
print("browser", browser["events"][0])
print("window", len(window["events"]))
print("afk", len(afk["events"]))
print("browser", len(browser["events"]))

# convert to numpy array
window_vocab = {}
for event in window["events"]:
  app_name = event["data"]["app"]
  if app_name not in window_vocab:
    window_vocab[app_name] = 0
  window_vocab[app_name] += 1
print("window_vocab", len(window_vocab))

url_vocab = {}
for event in browser["events"]:
  url_name = event["data"]["url"]
  if url_name not in url_vocab:
    url_vocab[url_name] = 0
  url_vocab[url_name] += 1
print("url_vocab", len(url_vocab))

afk_vocab = {}
for event in afk["events"]:
  status_name = event["data"]["status"]
  if status_name not in afk_vocab:
    afk_vocab[status_name] = 0
  afk_vocab[status_name] += 1
print("afk_vocab", len(afk_vocab))

filtered_url_vocab = [url for url in url_vocab.keys() if url_vocab[url] > 10] 
print(len(filtered_url_vocab))

sorted_window_vocab = sorted(window_vocab.items(), key= lambda x: x[1], reverse=True)
window_word_to_id = {sorted_window_vocab[i][0]: i for i in range(len(sorted_window_vocab))}
print("window_word_to_id", window_word_to_id)
print(sorted_window_vocab)

sorted_url_vocab = sorted(url_vocab.items(), key= lambda x: x[1], reverse=True)
url_word_to_id = {sorted_url_vocab[i][0]: i for i in range(len(sorted_url_vocab))}

window_data = []
for event in window["events"]:
  # print("event", event)
  app_id = window_word_to_id[event["data"]["app"]] 
  time = event["timestamp"]
  duration = event["duration"]
  # print("duration: {}, event {}".format(duration, event))
  date_time_obj = datetime.fromisoformat(time)
  timestamp = date_time_obj.timestamp()
  item = [timestamp, duration, 0, app_id]
  # window {'timestamp': '2021-04-08T01:44:56.409000+00:00', 'duration': 228.513, 'data': {'app': 'Google Chrome'}}
  window_data.append(item)
# print("window_vocab", window_vocab)
# print("window_vocab shape", np.array(window_data).shape)

url_data = []
for event in browser["events"]:
  # print("event", event)
  url_id = url_word_to_id[event["data"]["url"]]
  tab_count = event["data"]["tabCount"]
  audible = event["data"]["audible"]                                               
  incognito = event["data"]["incognito"]                                               
  time = event["timestamp"]
  date_time_obj = datetime.fromisoformat(time)
  timestamp = date_time_obj.timestamp()
  duration = event["duration"]
  item = [timestamp, duration, 1, url_id]
  # item = np.array([timestamp, duration, 1, url_id, audible, incognito, tab_count])
  # window {'timestamp': '2021-04-08T01:44:56.409000+00:00', 'duration': 228.513, 'data': {'app': 'Google Chrome'}}
  url_data.append(item)
# print(np.array(url_data).shape)

afk_data = []
for event in afk["events"]:
  # print("event", event)
  afk_id = event["data"]["status"] == "afk"                                             
  time = event["timestamp"]
  date_time_obj = datetime.fromisoformat(time)
  timestamp = date_time_obj.timestamp()
  duration = event["duration"]
  item = [timestamp, duration, 2, afk_id]
  afk_data.append(item)
# print(np.array(afk_data).shape)


# print("stopwatch", stopwatch["events"][0])
  # print(type(afk[key]))
  # print(afk["events"][0])
# "id", "created", "name", "type", "client", "hostname", "events"
# "timestamp", "duration", "events", "current_window", "client"
# files_ds = tf.data.Dataset.from_generator(
#     lambda: get_files_dataset(drive, folderid),
#     output_signature=tf.TensorSpec(shape=(), dtype=tf.string))

# files_ds = files_ds.repeat().shuffle(10)

# tf.io.decode_json_example

bucket_key_dict {'afk': 'aw-watcher-afk', 'desktop': 'aw-watcher-window', 'browser': 'aw-watcher-web', 'stopwatch': None}
<class 'dict'>
dict_keys(['aw-watcher-window', 'aw-watcher-afk', 'aw-watcher-web', 'aw-watcher-stopwatch'])
window {'timestamp': '2021-04-08T01:44:56.409000+00:00', 'duration': 228.513, 'data': {'app': 'Google Chrome'}}
afk {'timestamp': '2021-04-08T01:10:45.510000+00:00', 'duration': 2280.741, 'data': {'status': 'not-afk'}}
browser {'timestamp': '2021-04-09T07:31:05.511000+00:00', 'duration': 0.0, 'data': {'url': 'colab.research.google.com', 'audible': False, 'incognito': False, 'tabCount': 2}}
window 2023
afk 624
browser 458173
window_vocab 12
url_vocab 1875
afk_vocab 2
539
window_word_to_id {'Google Chrome': 0, 'WhatsApp': 1, 'Preview': 2, 'zoom.us': 3, 'redacted_app_5': 4, 'Notes': 5, 'Sublime Text': 6, 'Photo Booth': 7, 'System Preferences': 8, 'TextEdit': 9, 'Archive Utility': 10, 'unknown': 11}
[('Google Chrome', 1004), ('WhatsApp', 463), ('Preview', 224), ('

In [ ]:
all_data = np.concatenate([window_data, url_data, afk_data], axis=0)
print(all_data.shape)
print(all_data[0])
py_sorted_data = sorted(all_data, key= lambda x: x[0])
sorted_data = np.array(py_sorted_data)
# sorted_data = all_data
zero_count = 0
zero_window_count = 0
zero_url_count = 0
zero_afk_count = 0

for row in sorted_data:
  if row[1] == 0:
    zero_count += 1 
    if row[2] == 0:
      zero_window_count += 1
    if row[2] == 1:
      zero_url_count += 1
    if row[2] == 2:
      zero_afk_count += 1

print("count", zero_count, zero_window_count, zero_url_count, zero_afk_count)
print("", min(sorted_data[:,1]))
print("sorted", sorted_data[100000:100010])
print("window", window_data[:10])
print("url", url_data[:10])
print("afk", afk_data[:10])

(460820, 4)
[1.6178463e+09 2.2851300e+02 0.0000000e+00 0.0000000e+00]
count 328431 156 328262 13
 0.0
sorted [[1.60891404e+09 4.00090000e+01 1.00000000e+00 2.00000000e+00]
 [1.60891410e+09 0.00000000e+00 1.00000000e+00 1.80000000e+01]
 [1.60891410e+09 6.00120000e+01 1.00000000e+00 2.00000000e+00]
 [1.60891416e+09 0.00000000e+00 1.00000000e+00 1.80000000e+01]
 [1.60891418e+09 2.00040000e+01 1.00000000e+00 2.00000000e+00]
 [1.60891421e+09 1.38000000e+00 1.00000000e+00 0.00000000e+00]
 [1.60891421e+09 0.00000000e+00 1.00000000e+00 3.00000000e+00]
 [1.60891422e+09 0.00000000e+00 1.00000000e+00 2.00000000e+00]
 [1.60891422e+09 0.00000000e+00 1.00000000e+00 3.00000000e+00]
 [1.60891422e+09 9.99800000e+00 1.00000000e+00 0.00000000e+00]]
window [[1617846296.409, 228.513, 0, 0], [1617846270.616, 24.646, 0, 2], [1617846043.426, 225.988, 0, 0], [1617846030.499, 11.767, 0, 2], [1617844812.625, 1216.701, 0, 0], [1617844792.655, 18.781, 0, 2], [1617843205.909, 1585.57, 0, 0], [1617843185.198, 19.512

## Transformers

In [ ]:
!pip install git+https://github.com/deepmind/dm-haiku

  Cloning https://github.com/deepmind/dm-haiku to /tmp/pip-req-build-oomkk5dj
  Running command git clone -q https://github.com/deepmind/dm-haiku /tmp/pip-req-build-oomkk5dj
  Using cached https://files.pythonhosted.org/packages/ca/80/7c0cad11bd99985cfe7c09427ee0b4f9bd6b048bd13d4ffb32c6db237dfb/tabulate-0.8.9-py3-none-any.whl
  Created wheel for dm-haiku: filename=dm_haiku-0.0.4.dev0-cp37-none-any.whl size=545834 sha256=bd575a3ac552e57a3e42cbc2b6df00fbc829a7212b9ec2a655557b97618fb4d1
  Stored in directory: /tmp/pip-ephem-wheel-cache-ukqyh33t/wheels/97/0f/e9/17f34e377f8d4060fa88a7e82bee5d8afbf7972384768a5499
Successfully built dm-haiku
  Found existing installation: tabulate 0.8.3
    Uninstalling tabulate-0.8.3:
      Successfully uninstalled tabulate-0.8.3


In [ ]:
# Copyright 2020 DeepMind Technologies Limited. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Transformer model components."""

from typing import Optional

import haiku as hk
import jax
import jax.numpy as jnp
import numpy as np


class CausalSelfAttention(hk.MultiHeadAttention):
  """Self attention with a causal mask applied."""

  def __call__(
      self,
      query: jnp.ndarray,
      key: Optional[jnp.ndarray] = None,
      value: Optional[jnp.ndarray] = None,
      mask: Optional[jnp.ndarray] = None,
  ) -> jnp.ndarray:
    key = key if key is not None else query
    value = value if value is not None else query

    seq_len = query.shape[1]
    causal_mask = np.tril(np.ones((seq_len, seq_len)))
    mask = mask * causal_mask if mask is not None else causal_mask

    return super().__call__(query, key, value, mask)


class DenseBlock(hk.Module):
  """A 2-layer MLP which widens then narrows the input."""

  def __init__(self,
               init_scale: float,
               widening_factor: int = 4,
               name: Optional[str] = None):
    super().__init__(name=name)
    self._init_scale = init_scale
    self._widening_factor = widening_factor

  def __call__(self, x: jnp.ndarray) -> jnp.ndarray:
    hiddens = x.shape[-1]
    initializer = hk.initializers.VarianceScaling(self._init_scale)
    x = hk.Linear(self._widening_factor * hiddens, w_init=initializer)(x)
    x = jax.nn.gelu(x)
    return hk.Linear(hiddens, w_init=initializer)(x)


class Transformer(hk.Module):
  """A transformer stack."""

  def __init__(self,
               num_heads: int,
               num_layers: int,
               dropout_rate: float,
               name: Optional[str] = None):
    super().__init__(name=name)
    self._num_layers = num_layers
    self._num_heads = num_heads
    self._dropout_rate = dropout_rate

  def __call__(self,
               h: jnp.ndarray,
               mask: Optional[jnp.ndarray],
               is_training: bool) -> jnp.ndarray:
    """Connects the transformer.
    Args:
      h: Inputs, [B, T, H].
      mask: Padding mask, [B, T].
      is_training: Whether we're training or not.
    Returns:
      Array of shape [B, T, H].
    """

    init_scale = 2. / self._num_layers
    dropout_rate = self._dropout_rate if is_training else 0.
    if mask is not None:
      mask = mask[:, None, None, :]

    # Note: names chosen to approximately match those used in the GPT-2 code;
    # see https://github.com/openai/gpt-2/blob/master/src/model.py.
    for i in range(self._num_layers):
      h_norm = layer_norm(h, name=f'h{i}_ln_1')
      h_attn = CausalSelfAttention(
          num_heads=self._num_heads,
          key_size=64,
          w_init_scale=init_scale,
          name=f'h{i}_attn')(h_norm, mask=mask)
      h_attn = hk.dropout(hk.next_rng_key(), dropout_rate, h_attn)
      h = h + h_attn
      h_norm = layer_norm(h, name=f'h{i}_ln_2')
      h_dense = DenseBlock(init_scale, name=f'h{i}_mlp')(h_norm)
      h_dense = hk.dropout(hk.next_rng_key(), dropout_rate, h_dense)
      h = h + h_dense
    h = layer_norm(h, name='ln_f')

    return h


def layer_norm(x: jnp.ndarray, name: Optional[str] = None) -> jnp.ndarray:
  """Apply a unique LayerNorm to x with default settings."""
  return hk.LayerNorm(axis=-1,
                      create_scale=True,
                      create_offset=True,
                      name=name)(x)


In [ ]:
!pip install optax

In [ ]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / jnp.power(10000, (2 * (i//2)) / jnp.float32(d_model))
  return pos * angle_rates

def positional_encoding(position, d_model):
  angle_rads = get_angles(jnp.arange(position)[:, jnp.newaxis],
                          jnp.arange(d_model)[jnp.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = jnp.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = jnp.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[jnp.newaxis, ...]

  return pos_encoding

In [ ]:
"""Train a transformer for language modeling on LM1B.
This example serves to demonstrate:
  - A clean Haiku transformer implementation.
  - An example minimal training loop around it.
We have not tuned the hyperparameters for LM1B at all.
Note: Run with --alsologtostderr to see outputs.
"""

import functools
import os
import pickle
import time
from typing import Any, Mapping

from absl import app
from absl import flags
from absl import logging
import haiku as hk
from examples.transformer import dataset
from examples.transformer import model
import jax
import jax.numpy as jnp
import numpy as np
import optax

batch_size = 16
sequence_length = 128

d_model = 256
num_heads = 4
num_layers = 6
dropout_rate = 0.1

learning_rate = 2e-4
grad_clip_value = 0.25

checkpoint_dir = '/tmp/haiku-lm1b'

LOG_EVERY = 50
MAX_STEPS = 10**6


def build_forward_fn(activity_size: int, 
                     duration_size: int,
                     delay_size: int,
                     d_model: int, 
                     num_heads: int,
                     num_layers: int, 
                     dropout_rate: float):
  """Create the model's forward pass."""

  def forward_fn(data: Mapping[str, jnp.ndarray],
                 is_training: bool = True) -> jnp.ndarray:
    """Forward pass."""
    activity_tokens = data['activity_obs']
    duration_tokens = data['duration_obs']
    delay_tokens = data['delay_obs']
    input_mask = jnp.ones_like(activity_tokens)
    # duration_input_mask = jnp.ones_like(duration_tokens)
    # delay_input_mask = jnp.ones_like(delay_tokens)
    seq_length = activity_tokens.shape[1]

    # Embed the input tokens and positions.
    embed_init = hk.initializers.TruncatedNormal(stddev=0.02)
    activity_token_embedding_map = hk.Embed(activity_size, d_model, 
                                            w_init=embed_init)
    duration_token_embedding_map = hk.Embed(duration_size, d_model, 
                                            w_init=embed_init)
    delay_token_embedding_map = hk.Embed(duration_size, d_model, 
                                            w_init=embed_init)
    activity_token_embs = activity_token_embedding_map(activity_tokens)
    duration_token_embs = duration_token_embedding_map(duration_tokens)
    delay_token_embs = delay_token_embedding_map(delay_tokens)
    # TODO: get positional_embeddings from timestamp observation
    # positional_embeddings = positional_encoding(data['time_obs'], 60*60*24)
    # positional_embeddings = hk.get_parameter(
        # 'pos_embs', [seq_length, d_model], init=embed_init)
    input_embeddings = activity_token_embs + duration_token_embs + delay_token_embs 
    # +positional_embeddings

    # Run the transformer over the inputs.
    transformer = model.Transformer(
        num_heads=num_heads, num_layers=num_layers, dropout_rate=dropout_rate)
    output_embeddings = transformer(input_embeddings, input_mask, is_training)

    activity_logits = hk.Linear(activity_size)(output_embeddings)
    duration_logits = hk.Linear(duration_size)(output_embeddings)
    delay_logits = hk.Linear(delay_size)(output_embeddings)
    # Reverse the embeddings (untied).
    return activity_logits, duration_logits, delay_logits

  return forward_fn


def lm_loss_fn(forward_fn,
               activity_size: int,
               duration_size: int,
               delay_size: int,
               params,
               rng,
               data: Mapping[str, jnp.ndarray],
               is_training: bool = True) -> jnp.ndarray:
  """Compute the loss on data wrt params."""
  activity_logits, duration_logits, delay_logits  = forward_fn(params, rng, data, is_training)
  activity_targets = jax.nn.one_hot(data['activity_target'], activity_size)
  duration_targets = jax.nn.one_hot(data['duration_target'], duration_size)
  delay_targets = jax.nn.one_hot(data['delay_target'], delay_size)
  assert activity_logits.shape == activity_targets.shape
  assert duration_logits.shape == duration_targets.shape
  assert delay_logits.shape == delay_targets.shape, (delay_logits.shape, delay_targets.shape)

  
  mask = jnp.ones_like(data["activity_obs"])
  # duration_mask = jnp.greater(data['duration_obs'], 0)
  # delay_mask = jnp.greater(data['delay_obs'], 0)
  activity_loss = -jnp.sum(activity_targets * 
                           jax.nn.log_softmax(activity_logits), axis=-1)
  activity_loss = jnp.sum(activity_loss * 
                          mask) / jnp.sum(mask)
  duration_loss = -jnp.sum(duration_targets * 
                           jax.nn.log_softmax(duration_logits), axis=-1)
  duration_loss = jnp.sum(duration_loss * 
                          mask) / jnp.sum(mask)
  delay_loss = -jnp.sum(delay_targets * 
                        jax.nn.log_softmax(delay_logits), axis=-1)
  delay_loss = jnp.sum(delay_loss * mask) / jnp.sum(mask)
  return activity_loss + duration_loss + delay_loss


class Updater:
  """A stateless abstraction around an init_fn/update_fn pair.
  This extracts some common boilerplate from the training loop.
  """

  def __init__(self, net_init, loss_fn,
               optimizer: optax.GradientTransformation):
    self._net_init = net_init
    self._loss_fn = loss_fn
    self._opt = optimizer

  @functools.partial(jax.jit, static_argnums=0)
  def init(self, master_rng, data):
    """Initializes state of the updater."""
    out_rng, init_rng = jax.random.split(master_rng)
    params = self._net_init(init_rng, data)
    opt_state = self._opt.init(params)
    out = dict(
        step=np.array(0),
        rng=out_rng,
        opt_state=opt_state,
        params=params,
    )
    return out

  @functools.partial(jax.jit, static_argnums=0)
  def update(self, state: Mapping[str, Any], data: Mapping[str, jnp.ndarray]):
    """Updates the state using some data and returns metrics."""
    rng, new_rng = jax.random.split(state['rng'])
    params = state['params']
    loss, g = jax.value_and_grad(self._loss_fn)(params, rng, data)

    updates, opt_state = self._opt.update(g, state['opt_state'])
    params = optax.apply_updates(params, updates)

    new_state = {
        'step': state['step'] + 1,
        'rng': new_rng,
        'opt_state': opt_state,
        'params': params,
    }

    metrics = {
        'step': state['step'],
        'loss': loss,
    }
    return new_state, metrics


class CheckpointingUpdater:
  """A didactic checkpointing wrapper around an Updater.
  A more mature checkpointing implementation might:
    - Use np.savez() to store the core data instead of pickle.
    - Not block JAX async dispatch.
    - Automatically garbage collect old checkpoints.
  """

  def __init__(self,
               inner: Updater,
               checkpoint_dir: str,
               checkpoint_every_n: int = 10000):
    self._inner = inner
    self._checkpoint_dir = checkpoint_dir
    self._checkpoint_every_n = checkpoint_every_n

  def _checkpoint_paths(self):
    return [p for p in os.listdir(self._checkpoint_dir) if 'checkpoint_' in p]

  def init(self, rng, data):
    """Initialize experiment state."""
    if not os.path.exists(self._checkpoint_dir) or not self._checkpoint_paths():
      os.makedirs(self._checkpoint_dir, exist_ok=True)
      return self._inner.init(rng, data)
    else:
      checkpoint = os.path.join(self._checkpoint_dir,
                                self._checkpoint_paths()[-1])
      logging.info('Loading checkpoint from %s', checkpoint)
      with open(checkpoint, 'rb') as f:
        state = pickle.load(f)
      return state

  def update(self, state, data):
    """Update experiment state."""
    # NOTE: This blocks until `state` is computed. If you want to use JAX async
    # dispatch, maintain state['step'] as a NumPy scalar instead of a JAX array.
    # Context: https://jax.readthedocs.io/en/latest/async_dispatch.html
    step = np.array(state['step'])
    if step % self._checkpoint_every_n == 0:
      path = os.path.join(self._checkpoint_dir,
                          'checkpoint_{:07d}.pkl'.format(step))
      checkpoint_state = jax.device_get(state)
      logging.info('Serializing experiment state to %s', path)
      with open(path, 'wb') as f:
        pickle.dump(checkpoint_state, f)

    state, out = self._inner.update(state, data)
    return state, out


##

In [ ]:
import tensorflow as tf
max_afk_vocab_size = 2
max_window_vocab_size = 100
max_url_vocab_size = 5000

combined_data = []
for row in sorted_data:
  if row[2] == 0: # window
    rename_vocab = row[3] + max_afk_vocab_size
  elif row[2] == 1: # url
    rename_vocab = row[3] + max_afk_vocab_size + max_window_vocab_size
  elif row[2] == 2:
    rename_vocab = row[3]
  else:
    raise
  if row[3] == 1874:
    print("rename_vocab", rename_vocab, row[3], row[2])
  combined_data.append([row[0], row[1], rename_vocab])
np_combined = np.array(combined_data)

rename_vocab 1976.0 1874.0 1.0


In [ ]:
print(sorted_data.shape)
print(np_combined.shape)
print("max", np.max(np_combined[:,2]))
print(len(url_vocab))
print(len(window_vocab))
np_url = np.array(url_data)
print("np_url", np.max(np_url[:,3]))
np_all = np.array(sorted_data)
print("np_all shape", np_all.shape)
print("np_all", np.max(np_all[:,3]))

(460820, 4)
(460820, 3)
max 1976.0
1875
12
np_url 1874.0
np_all shape (460820, 4)
np_all 1874.0


## tdfs

In [ ]:
import tensorflow_datasets as tfds

# fix preprocessing: use window instead of batch
timestamps = np_combined[:, 0]
# timestamps = [t1, t2, ..]
# delays = [t2-t1, t3-t2, ..., t[n]-t[n-1]]
# [t2, t3, t4, ..., tn] - [t1, t2, ..., t[n-1] ]
# timestamps[1:] - timestamps[:-1]
delays = timestamps[1:] - timestamps[:-1]
zero = np.zeros([1])
all_delay = np.concatenate([zero, delays], axis = 0) 
delay_added = np.concatenate([np_combined, all_delay[:,None]], axis = 1)
tf_data_list = tf.data.Dataset.from_tensor_slices(delay_added).batch(64, drop_remainder=True).repeat().shuffle(1600).batch(16, drop_remainder=True)
# train_dataset = tf_data
tf_data = tf_data_list.map(lambda x: {"activity_obs": tf.cast(x[:,:-1,2], tf.int32), 
                                      "duration_obs": tf.cast(x[:,:-1,1], tf.int32),
                                      "time_obs": tf.cast(x[:,:-1,0], tf.int32),
                                      "delay_obs": tf.cast(x[:,:-1,3], tf.int32),
                                      "activity_target": tf.cast(x[:,1:,2], tf.int32),
                                      "duration_target": tf.cast(x[:,1:,1], tf.int32),
                                      "delay_target": tf.cast(x[:,1:,3], tf.int32)})
# tf_data = tf_data_list.map(lambda x: {"obs": tf.cast(x[:,:-1,2], tf.int32), 
                                      # "target": tf.cast(x[:,1:,2], tf.int32)})
# print(next(iter(tf_data)).shape)
train_dataset = iter(tfds.as_numpy(tf_data))

In [ ]:
# ds = load(16,64)
# next(ds).shape

## train & evaluate model

In [ ]:
vocab_size = 2000
activity_size = 2000
duration_size = 5
delay_size = 5
# Set up the model, loss, and updater.
forward_fn = build_forward_fn(activity_size, duration_size, delay_size,
                              d_model, num_heads, num_layers, dropout_rate)
forward_fn = hk.transform(forward_fn)
loss_fn = functools.partial(lm_loss_fn, forward_fn.apply, 
                            activity_size, duration_size, delay_size)

optimizer = optax.chain(
    optax.clip_by_global_norm(grad_clip_value),
    optax.adam(learning_rate, b1=0.9, b2=0.99))

updater = Updater(forward_fn.init, loss_fn, optimizer)
# updater = CheckpointingUpdater(updater, checkpoint_dir)

# Initialize parameters.
logging.info('Initializing parameters...')
rng = jax.random.PRNGKey(428)
data = next(train_dataset)
state = updater.init(rng, data)

logging.info('Starting train loop...')
prev_time = time.time()
for step in range(MAX_STEPS):
  data = next(train_dataset)
  state, metrics = updater.update(state, data)
  # We use JAX runahead to mask data preprocessing and JAX dispatch overheads.
  # Using values from state/metrics too often will block the runahead and can
  # cause these overheads to become more prominent.
  if step % LOG_EVERY == 0:
    steps_per_sec = LOG_EVERY / (time.time() - prev_time)
    prev_time = time.time()
    metrics.update({'steps_per_sec': steps_per_sec})
    logging.info({k: float(v) for k, v in metrics.items()})

Streaming output truncated to the last 5000 lines.
INFO - {'loss': 0.1620626449584961, 'step': 750000.0, 'steps_per_sec': 57.93543250486421}
INFO - {'loss': 0.1536264568567276, 'step': 750050.0, 'steps_per_sec': 57.65930631286526}
INFO - {'loss': 0.12423676252365112, 'step': 750100.0, 'steps_per_sec': 57.481038388082084}
INFO - {'loss': 0.13355135917663574, 'step': 750150.0, 'steps_per_sec': 57.67086539902624}
INFO - {'loss': 0.12135814130306244, 'step': 750200.0, 'steps_per_sec': 57.5952999929144}
INFO - {'loss': 0.1370733380317688, 'step': 750250.0, 'steps_per_sec': 57.8089487941354}
INFO - {'loss': 0.14799010753631592, 'step': 750300.0, 'steps_per_sec': 57.68550721607969}
INFO - {'loss': 0.13761937618255615, 'step': 750350.0, 'steps_per_sec': 57.58299642720247}
INFO - {'loss': 0.14659112691879272, 'step': 750400.0, 'steps_per_sec': 57.68879193790858}
INFO - {'loss': 0.1383853405714035, 'step': 750450.0, 'steps_per_sec': 57.509364132937094}
INFO - {'loss': 0.17009492218494415, 'step'